# Data processing from raw
BAUDIER Stéfan, FRERING Arnaud, JUPPET Alexandre, KNELL Nicolas, RODRIGUEZ Thomas

##Libraries import

In [1]:

import pandas as pd
import math as m
import numpy as np
import datetime
import time
from sklearn.model_selection import train_test_split
from tqdm import trange

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  print("Not using COLAB")
  pass

import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import optimizers, losses
import tensorflow_datasets as tfds

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

print("TensorFlow version:",tf.__version__)
tf.test.gpu_device_name()

TensorFlow 2.x selected.


Using TensorFlow backend.


TensorFlow version: 2.0.0


'/device:GPU:0'

##Data import

In [0]:

raw_train = pd.read_csv("https://raw.githubusercontent.com/AlexandreJup/Hackathon/master/train.csv",sep=',')
raw_test = pd.read_csv("https://raw.githubusercontent.com/AlexandreJup/Hackathon/master/test_student.csv",sep=',')

##Data modification

In [0]:
mod_raw_train = raw_train.drop(columns=['from','to', 'index', 'date'])
mod_raw_test = raw_test.drop(columns=['from','to', 'index', 'date'])

Fonctions


In [0]:
# Attention les deux df data_train et data_test doivent être donnés sans le(s) label(s)
# data_train correspond au pd.df d'entrainement (il faut que les normalisations, etc soient déjà faits)
# data_test correspond au pd.df de test (le vrai sans )
# column_keep est une liste de string des noms des colonnes à garder
# column_dummies est une liste de string des noms des colonnes à transformer en 1-hot.
# cette fonction renvoie 2 liste : l'entrainement et le test avec les bonnes colonnes et les 1-hot demandés

def concat_deconcat(data_train,data_test, column_keep, column_dummies):
  len_column_keep = len(column_keep)
  len_column_dummies = len(column_dummies)
  len_data_train = len(data_train)
  len_data_test = len(data_test)
  data_concat=pd.concat([data_train,data_test],sort=False)
  data = data_concat.loc[:,column_keep]
  data = pd.get_dummies(data,columns = column_dummies)
  modified_train = data.iloc[:len_data_train]
  modified_test = data.iloc[len_data_train:]
  return (modified_train, modified_test)

def send_to_csv(dataframe, file_name):
  dataframe.to_csv(index=False, path_or_buf= file_name)



def stacking_prep(list_of_models, list_of_files):
  l = len(list_of_files)
  list_of_df = [0 for _ in range(l)]
  
  for i in range(l):
    list_of_files[i] = "/content/drive/My Drive/Datasets Hackathon/"+list_of_files[i]
    list_of_df[i] = pd.read_csv(list_of_files[i])
  input_for_stacking = pd.concat(list_of_df, axis=1, sort=False)
  input_for_stacking = input_for_stacking.drop(columns = "Id")
  input_for_stacking.columns = list_of_models
  return(input_for_stacking)



Conversion of the scheduled_time in four informations:

*   The month
*   The day of the week
*   The hour
*   If the example is during weekend or not

In [5]:
nb_rows_train = len(mod_raw_train['scheduled_time'])
Month_train = [0 for i in range(nb_rows_train)]
Weekend_train = [0 for i in range(nb_rows_train)]
Hour_train = [0 for i in range(nb_rows_train)]
Day_train = [0 for i in range(nb_rows_train)]

nb_rows_test = len(mod_raw_test['scheduled_time'])
Month_test = [0 for i in range(nb_rows_test)]
Weekend_test = [0 for i in range(nb_rows_test)]
Hour_test = [0 for i in range(nb_rows_test)]
Day_test = [0 for i in range(nb_rows_test)]

for i in trange(nb_rows_train):
  date = mod_raw_train['scheduled_time'][i]

  # Defining the format of the date
  format_date = '%Y-%m-%d %H:%M:%S'
  datetime_obj = datetime.datetime.strptime(date, format_date)

  # Defining which days are weekends
  if datetime_obj.weekday() in [5, 6]:
    Weekend_train[i] = 1
  Month_train[i] = datetime_obj.month
  Hour_train[i] = datetime_obj.hour
  Day_train[i] = datetime_obj.weekday()

for i in trange(nb_rows_test):
  date = mod_raw_test['scheduled_time'][i]
  format_date = '%Y-%m-%d %H:%M:%S'
  datetime_obj = datetime.datetime.strptime(date, format_date)
  if datetime_obj.weekday() in [5, 6]:
    Weekend_test[i] = 1
  Month_test[i] = datetime_obj.month
  Hour_test[i] = datetime_obj.hour
  Day_test[i] = datetime_obj.weekday()

mod_raw_train['month'] = Month_train
mod_raw_train['hour'] = Hour_train
mod_raw_train['is_weekend'] = Weekend_train
mod_raw_train['day'] = Day_train

mod_raw_test['month'] = Month_test
mod_raw_test['hour'] = Hour_test
mod_raw_test['is_weekend'] = Weekend_test
mod_raw_test['day'] = Day_test

100%|██████████| 179350/179350 [00:04<00:00, 44415.07it/s]


## Normalisation of the data : (x-min)/(max-min)

In [0]:
#Normalisation
#stop_sequence
min_stop_sequence = min(min(mod_raw_train['stop_sequence']), min(mod_raw_test['stop_sequence']))
max_stop_sequence = max(max(mod_raw_train['stop_sequence']), max(mod_raw_test['stop_sequence']))

mod_raw_train['stop_sequence'] = (mod_raw_train['stop_sequence']-min_stop_sequence)/(max_stop_sequence-min_stop_sequence)
mod_raw_test['stop_sequence'] = (mod_raw_test['stop_sequence']-min_stop_sequence)/(max_stop_sequence-min_stop_sequence)

In [0]:
# Transforming the 'estimated' status to 'departed'
mod_raw_train.loc[mod_raw_train['status'] == 'estimated', 'status'] = 'departed'
mod_raw_test.loc[mod_raw_test['status'] == 'estimated','status'] = 'departed'

In [0]:
#Features and labels
feature_train = mod_raw_train.drop(columns=['delay_minutes'])
feature_test = mod_raw_test

# Defining the label 
label_train = mod_raw_train['delay_minutes']

## Création du dataframe final


In [0]:
# Use the 'concat_deconcat' function to choose which columns to keep and which ones to 1-hot encode
training_features, testing_features = concat_deconcat(feature_train, feature_test, ['day','stop_sequence','line', 'hour','from_id', 'to_id'],['day','line','from_id', 'to_id', 'hour'])

#Partie Apprentissage

##Partie Calcul RMSE

In [0]:
# Splitting the dataset into train and test subsets for training 
X_train, X_test, y_train, y_test = train_test_split(training_features, label_train, test_size=0.2,shuffle = False)

###Random forest

In [0]:
# create regressor object 
regressor = rf(n_estimators = 20, max_depth = 100, random_state = 0) 
  
# fit the regressor with x and y data 
regressor.fit(X_train, y_train)  

# Use the forest's predict method on the test data
predictions = regressor.predict(X_test)
# Calculate the absolute errors
nb_lines = len(X_test)
error = 0
for i in range(nb_lines):
  error+= (predictions[i]-y_test.iloc[i,].values)**2
error = np.sqrt(error/nb_lines)

# Print out the RMSE
print('RMSE:', error)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RMSE: [6.7272565]


### Random Forest split

In [0]:
# creating a random forest by splitting the dataset into subsets, separating by train line
motif = 'line'

list_motif_in_columns = []
for name_column in X_train:
  if motif in name_column:
    list_motif_in_columns.append(name_column)

In [0]:
nb_lines = len(list_motif_in_columns)
list_rmse = [0 for i in range(0,nb_lines)]

#Splitting the dataset into subsets of lines 
for i in range (0, nb_lines):
  print(i)
  targeted_train_lignes = X_train[list_motif_in_columns[i]]==1
  df_train_features_i = X_train[targeted_train_lignes]
  df_train_label_i = y_train[targeted_train_lignes]
  df_train_features_i = df_train_features_i.drop(columns = list_motif_in_columns)

  targeted_test_lignes = X_test[list_motif_in_columns[i]]==1
  df_test_features_i = X_test[targeted_test_lignes]
  df_test_label_i = y_test[targeted_test_lignes]
  df_test_features_i = df_test_features_i.drop(columns = list_motif_in_columns)

  # create regressor object 
  regressor = rf(n_estimators = 10, random_state = 0) 
  
  # fit the regressor with x and y data 
  regressor.fit(df_train_features_i, df_train_label_i)  

  # Use the forest's predict method on the test data
  predictions = regressor.predict(df_test_features_i)
  # Calculate the absolute errors
  rmse = sqrt(mean_squared_error(df_test_label_i, predictions))

  list_rmse[i] = rmse
  
print(list_rmse)
print(np.mean(list_rmse))

### MLP

In [0]:
# Defining the MLP model, we choose this architecture and tried several layers combination

class MLP(tf.keras.Model):

    def __init__(self, hidden_size, output_size):
        super().__init__()
        self.base_layer = tf.keras.layers.Dense(hidden_size, activation = 'relu')
        self.dense_layer = tf.keras.layers.Dense(hidden_size, activation = 'relu')
        self.regression = tf.keras.layers.Dense(output_size, activation='relu')
        self.dropout = tf.keras.layers.Dropout(rate=0.2)

    def call(self, x):
        x = self.base_layer(x)
        x = self.dense_layer(x)
        x = self.dense_layer(x)
        x = self.dense_layer(x)
        x = self.dropout(x)
        return self.regression(x)


model_MLP = MLP(hidden_size=64,output_size=1)

epochs = 25
batch_size = 500

optimizer = optimizers.Adam(learning_rate=1e-3)

#We monitor the mean_squared_error for training
loss = losses.mean_squared_error

metrics = ['mse']

model_MLP.compile(optimizer, loss=loss)

# Training our model on the splitted dataset
model_MLP.fit(X_train.values, y_train.values, batch_size=batch_size, epochs=epochs, validation_split=0.2)




In [24]:
# Evaluating our model by calculating the RMSE between y_pred and y_test
# We need to change the format of y_pred because the output is a ist and not an array
y_pred_train = model_MLP.predict(X_test)
nb_predicts = len(y_pred_training)
y_pred_training = [0 for i in range(nb_predicts)]
for i in range(nb_predicts):
  y_pred_training[i] = y_pred_train[i][0]
  

# final vector is y_pred_training
RMSE = np.sqrt(mean_squared_error(y_test.values, y_pred_training))
print(RMSE)


# storing the training results into a dataframe, used later
results_mlp_training = pd.DataFrame({'Id' : range(0,nb_predicts), "Predicted" : y_pred_training})

6.7541083255946


##Calcul des predictions

### Randomforest

In [0]:
regressor = rf(n_estimators = 20, random_state = 0) 
regressor.fit(training_features, label_train)

Y_pred = regressor.predict(testing_features)
Y_pred_train = regressor.predict(training_features)

results_rf_basic = pd.DataFrame({'Id' : range(0,len(Y_pred)), "Predicted" : Y_pred})
results_rf_basic_training = pd.DataFrame({'Id' : range(0,len(Y_pred)), "Predicted" : Y_pred_train})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


### Randomforest split

In [0]:
def compute_predictions_split(df_trainer, df_to_predict, motif):

  list_motif_in_columns = []
  for name_column in df_trainer:
    if motif in name_column:
      list_motif_in_columns.append(name_column)
  nb_lines = len(list_motif_in_columns)
  l = len(df_to_predict)
  id = [i for i in range(0,l)]
  init_pred = [-1 for i in range(0,l)]
  results = pd.DataFrame({ 'Id' : id, "Predicted" : init_pred })
  for i in trange (0, nb_lines):

    targeted_train_lignes = df_trainer[list_motif_in_columns[i]]==1
    df_train_features_i = df_trainer[targeted_train_lignes]
    df_train_label_i = label_train[targeted_train_lignes]
    df_train_features_i = df_train_features_i.drop(columns = list_motif_in_columns)

    targeted_test_lignes = df_to_predict[list_motif_in_columns[i]]==1
    df_test_features_i = df_to_predict[targeted_test_lignes]
    df_test_features_i = df_test_features_i.drop(columns = list_motif_in_columns)

    # create regressor object 
    regressor = rf(n_estimators = 20, random_state = 0) 
    
    # fit the regressor with x and y data 
    regressor.fit(df_train_features_i, df_train_label_i)  

    # Use the forest's predict method on the test data
    predictions = regressor.predict(df_test_features_i)
    
    results.loc[targeted_test_lignes, "Predicted"] = predictions
  return(results)

In [0]:
results_rf_line_split = compute_predictions_split(training_features, testing_features, "line")
results_rf_day_split = compute_predictions_split(training_features, testing_features, "day")

results_rf_line_split_training = compute_predictions_split(training_features, training_features, "line")
results_rf_day_split_training = compute_predictions_split(training_features, training_features, "day")

###MLP


In [27]:
# Predicting the delays on the real test dataset (and transforming the y_pred to the right format)
y_pred_test = model_MLP.predict(testing_features)
nb_lines = len(testing_features)
y_pred_testing = [0 for i in range(nb_lines)]
for i in range(nb_lines):
  y_pred_testing[i] = y_pred_test[i][0]

# storing the results in a dataframe
results_mlp = pd.DataFrame({'Id' : range(0,nb_lines), "Predicted" : y_pred_testing})

###KNN

In [0]:
motif_1 = 'day'
motif_2 = 'line'

list_motif_1_in_columns = []
for name_column in training_features:
  if motif_1 in name_column:
    list_motif_1_in_columns.append(name_column)

list_motif_2_in_columns = []
for name_column in training_features:
  if motif_2 in name_column:
    list_motif_2_in_columns.append(name_column)

nb_motifs_1 = len(list_motif_1_in_columns)
nb_motifs_2 = len(list_motif_2_in_columns)

l = len(training_features)
id = [i for i in range(0,l)]
init_pred = [-1 for i in range(0,l)]
results_KNN = pd.DataFrame({ 'Id' : id, "Predicted" : init_pred })

for i in trange (0, nb_motifs_1):
  for j in trange (0, nb_motifs_2):
    
    #Select the lignes in the train df with day==i and line==j
    targeted_train_lignes = (training_features[list_motif_1_in_columns[i]]!=0) & (training_features[list_motif_2_in_columns[j]]!=0)
    df_train_features_ij = training_features[targeted_train_lignes]
    df_train_label_ij = label_train[targeted_train_lignes]
    #Drop the line and day columns from the choosen train dataset
    df_train_features_ij = df_train_features_ij.drop(columns = list_motif_1_in_columns)
    df_train_features_ij = df_train_features_ij.drop(columns = list_motif_2_in_columns)

    #Select the lignes in the test df with day==i and line==j
    targeted_test_lignes = (training_features[list_motif_1_in_columns[i]]!=0) & (training_features[list_motif_2_in_columns[j]]!=0)
    df_test_features_ij = training_features[targeted_test_lignes]
    #Drop the line and day columns from the choosen test dataset
    df_test_features_ij = df_test_features_ij.drop(columns = list_motif_1_in_columns)
    df_test_features_ij = df_test_features_ij.drop(columns = list_motif_2_in_columns)

    if ( (len(df_train_features_ij)!=0) & (len(df_test_features_ij)!=0)):
      # create regressor object 
      regressor = rf(n_estimators = 10, random_state = 0) 
      
      # fit the regressor with x and y data 
      regressor.fit(df_train_features_ij, df_train_label_ij)  

      # Use the forest's predict method on the test data
      predictions = regressor.predict(df_test_features_ij)
      
      results_KNN.loc[targeted_test_lignes, "Predicted"] = predictions
    else:
       if (len(df_train_features_ij)==0):
         print('il manque dans le data train:')
         print(list_motif_1_in_columns[i],list_motif_2_in_columns[j])
       if (len(df_test_features_ij)==0):
         print('il manque dans le data test:')
         print(list_motif_1_in_columns[i],list_motif_2_in_columns[j])

###Stack the 4 methods

In [0]:
results_stacking = pd.DataFrame({"Predicted_basic" : results_rf_10_basic['Predicted'], "Predicted_line_split" : results_rf_10_line_split['Predicted'], "Predicted_day_split" : results_rf_10_day_split['Predicted'], "Predicted_MLP"  : results_mlp['Predicted']})
results_stacking_training = pd.DataFrame({"Predicted_basic" : results_rf_10_basic_training['Predicted'], "Predicted_line_split" : results_rf_10_line_split_training['Predicted'], "Predicted_day_split" : results_rf_10_day_split_training['Predicted'], "Predicted_MLP"  : results_mlp_training['Predicted']})

Our best results were with the simplest meta classifier : the average

In [0]:
predictions_av = (results_stacking['Predicted_basic']+results_stacking['Predicted_line_split']+results_stacking['Predicted_day_split']+results_stacking["Predicted_MLP"])/4
results_av = pd.DataFrame({'Id' : range(0,len(predictions_av)), "Predicted" : predictions_av})

But we also tried with more complex meta-classifiers

#####Linear regressor

In [0]:
regressor = LinearRegression().fit(results_stacking_training, label_train)
predictions_lr = regressor.predict(results_stacking)
results_lr = pd.DataFrame({'Id' : range(0,len(predictions_lr)), "Predicted" : predictions_lr})

####KNN

In [0]:
regressor = KNeighborsRegressor(n_neighbors=5).fit(results_stacking_training, label_train)
predictions_knn = regressor.predict(results_stacking)
results_knn = pd.DataFrame({'Id' : range(0,len(predictions_knn)), "Predicted" : predictions_knn})

####RandomForest

In [0]:
regressor = rf(n_estimators = 20, random_state = 0).fit(results_stacking_training, label_train)
predictions_rf = regressor.predict(results_stacking)
results_rf = pd.DataFrame({'Id' : range(0,len(predictions_rf)), "Predicted" : predictions_rf})

#Hackathon report
BAUDIER Stéfan, FRERING Arnaud, JUPPET Alexandre, KNELL Nicolas, RODRIGUEZ Thomas




###1) The preprocessing


For the preprocessing part, we made differents modifications to optimize our datasets.
First, we removed six columns : from, to, `index`, `date`, `status` and `train_id`.
From and to are replaced by the `from_id` and `to_id`.
The index column doesn’t give us important information, as id is giving the same.
The `date` is replaced by the `scheduled_time` because we can get the same information.
The `status` of the train wasn’t leading to better results, we deduced that it was a source of overfitting so we removed it.
The `train_id` needs to be transformed into 1-hot encoded but this operation adds too much columns and leads to overfitting.

After that, we transformed `scheduled_time` into `day` and `hour` columns by using the datetime library. We tried to use the month as column but as the months during the training and the testing dataset are different, the generalisation is bad.

Then we have normalized the stop-sequence column.

We encoded the columns `from_id`, `to_id`, `day` and `hour` in one hot vectors. The idea was to applied that on the test and train datasets combined. By doing this method on the two datasets combined, we added the missing information from each dataset in the other one. For example, if a line is present in the testing dataset but not in the training one, we need the column of this line in order to build its one hot vector and its prediction.

###2) The approaches : how the best parameters were selected

For the random forest, we tried with 1,5, 10, 20, 50 and 100 trees. The improvement for the RMSE were not very convincing with many trees compared to the high computation time, so we chose to keep 20 trees.
For the maximum depth of a tree, we chose 100 in order to get a satisfying RMSE.

###3) A comparison of the methods and their results

We tested several models, in a first approach we used simple models of machine learning in order to see their results.

Linear regression : **score = 102894604476**

Tree : **score =  8.839** (max depth of 100)

kNN : **score = 8.586** (5 neighbours)

The prediction of the linear regression was very bad and predicted some very high numbers, which is why the RMSE is exploding. For the tree and for kNN we get reasonable results, because they are more efficient for modelling complex phenomena.

Those results are comforting us in the idea that complex models will have better results. The idea was then to try a deep model so we firstly thought of a RNN. We didn’t succeed in implementing it, so we made a MLP with 3 hidden layers, ReLu as the activation function and a layer of dropout in order to avoid the overfitting.

MLP : **score = 8.271**

This prediction is way better than the simple models but not enough satisfying yet.

In order to improve the performance, we implemented some ensemble learning methods. First, a random forest composed of 20 trees and with a max depth of 100. For this model, we tried many configurations of datasets (only the lines as one hot vectors, only the days as one hot vectors and the whole dataset which had the best result presented here). We then wanted to compile those models in a stacking approach, with the 3 random forest as primary predictors and an average function as meta-classifier.

**Random forest : 8.208**

**Stacking : 8.182**
>
Those results were the best we could get even if we tried other methods like bagging or RNN that didn’t worked well.

###4) Our group dynamic

We worked on a shared Google Colab that was gathering our progress while everyone had a personal copy in order to try new things. Each time somebody found a good method or improved our model and datasets, he push the modifications on the shared file.

How took place our collabotary work ?
At the beginning we discussed about the dataset to try to improve it for a future use. Like a debate, we discussed about all the different ideas on a board and arrived to a consensus. 
Once the preprocessing had ended, each of us was in charge of a specific model. The idea was to split the work to test all the models we saw during lessons. After computing different methods, we kept the most accurate one, the random forest, and we tried to improve it with some ensemble learning methods. Our best result was obtained with the stacking of random forest.

###5) Difficulties encountered

The construction of the RNN and the MLP were difficult to implement for the data processing.
Once the **8.18** score of RMSE was reached, it was difficult for us to have a real improvement despite our data processing and our model improvements. This was a result of finding new ideas once we tried all the models we know and use ensemble learning to improve them. The difficulty to reach a better RMSE can be also linked to our preprocessing and the choices we made.


